# Web Scraping: Extrayendo datos de Internet

## Introducción a HTML

El lenguaje principal de la internet es HTML, cuando nosotros vemos algo así

![](https://github.com/institutohumai/cursos-python/blob/master/Scraping/1_HTTP_Inicial/multimedia/hello-world.jpeg?raw=1)

Eso se genera a partir de una código que luce así

```
<html>
<header><title>Web Scraping - Instituto Humai</title></header>
<body>
<h1>¡Hola!</h1>
<p>Esto es un sitio web</p>
</body>
</html>
```

**_Nota_**: Para saber más sobre HTML podés consultar [acá](https://www.w3schools.com/TAGS/default.ASP) la lista de etiquetas de este lenguaje.

## ¿Cómo consigo el código HTML?

Ahora que sabemos cuál es el componente principal de los sitios webs podemos intentar programar a nuestra computadora para leer HTML y extraer información útil.

Para conseguir el código de un sitio web podemos presionar ctrl+u en el navegador.

Para hacer lo mismo desde python se hace lo siguiente

In [ ]:
#Importamos la libreria necesaria
import requests

un_sitio_web = "https://es.wikipedia.org/wiki/HTML"

# esto descarga la información del sitio web
# similarmente a lo que hace un navegador web antes de mostrarnos el contenido de forma mas amigable para un humano
resultado = requests.get(un_sitio_web)

# accedemos al código yendo al atributo "text" del resultado
codigo_html = resultado.text
#codigo_html

### ¿Qué acabo de hacer?

Veamos algunos detalles más sobre cómo descargar el contenido de un sitio web (O cómo se le suele decir en la jerga de la programación _realizar un request_). Como dijimos, en python se puede utilizar la función get de la libreria requests para hacer esto, veamos con mayor profundidad cómo se utiliza.

In [ ]:
url = 'http://www.laprensa.com.ar/'

headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
resp = requests.get(url, headers = headers)

A parte de la _url_, muchas veces se especifican los _headers_, estos son objetos que proveen datos sobre nuestro _request_, por ejemplo en el campo user-agent brindamos detalles sobre quienes somos (Nuestro sistema operativo, navegador web y demás). En este caso, como no estamos usando un navegador sino que hacemos el _request_ desde Python normalmente se omite este campo, o en caso de ser obligatorio se puede inventar, ya que algunos sitios nos van a ignorar a menos que especifiquemos este campo.

- Consultas
    - ¿Por qué los sitios te podrían bloquear/ignorar?
    - ¿De donde saco un user-agent?

Como vimos antes la función get retorna un objeto, el cual llamamos _resp_, este es un elemento de la clase _Response_ y tiene distintos atributos a los que podemos acceder.

In [ ]:
#Vemos el código de estado
# 200 es que esta todo bien, 5xx o 4xx es que esta todo mal (Por ejemplo el clasico 404)
resp.status_code

200

In [ ]:
#Vemos los headers que enviamos
resp.request.headers

{'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

El atributo que nos interesa particularmente es resp.text, que guardan el contenido de la página.

Como vamos a descargar el codigo de un sitio frecuentemente armamos una funcion para no reescribir lo mismo muchas veces

In [ ]:
def codigo_html(url):
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
    resp = requests.get(url, headers = headers)
    return resp.text

### Documentación
La función get y la clase Response fueron desarrolladas por lxs programadores que crearon la librería requests. Si quieren saber mas sobre algún detalle siempre es recomendable buscar en la documentación oficial de la librería.

## ¿Cómo extraigo datos útiles del código HTML?

- Veamos un ejemplo inspeccionando con chrome un sitio web


### Método 1: Expresiones regulares

RegEx para los amigos. Son un mini lenguaje de programación diseñado para realizar búsquedas en strings.

Las funciones principales de la librería re son:
- re.findall(pattern, string) para encontrar todos los resultados de una búsqueda
- re.search(pattern, string) para encontrar el primer resultado que coincida
- re.sub(pattern, replace, string) para substituir un texto por otro

#### Recursos útiles

- [Testeo de regex online](https://regex101.com/)
- [CheatSheet](https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf)



<h2><center>Sintaxis para construir regex</center></h2>


<h3><center>Grupos de captura</center></h3>


|     |                       |
|-----|-----------------------|
| ()  | grupo de captura      |
|(?:) | grupo de no captura   |

<h3><center>Tipos de datos</center></h3>


|     |                      |          |                         |
|----|-----------------------|----------|-------------------------|
| \w | caracter alfanumérico | .        | cualquier cosa menos \n |
| \d | dígito                | \|       | operador "or"           |
| \s | espacio en blanco     | [m-z3-9] | rangos                  |

<h3><center>Operadores</center></h3>

|         |                      |
|---------|----------------------|
| \|      | operador "or"        |
| []      | conjunto             |
|[m-z3-9] | rangos               |


<h3><center>Cuantificadores</center></h3>

|      |                                              |
|------|----------------------------------------------|
| +    | Uno o más del elemento anterior              |
| *    | Cero o más del elemento anterior             |
| {4,} | Cuatro o más del elemento anterior           |
| ?    | Cambia el operador anterior de lazy a greedy |

### ¿Cómo se usa eso? Veamos ejemplos

In [ ]:
import re

# a- extraer números de una oración.

texto = "Mi nombre es Mathias y mi teléfono es 1564232324"
regla_de_busqueda = "15\d+"
print(re.findall(regla_de_busqueda,texto))

# En realidad los telefonos no son cualquier seguidilla de numeros
# suelen tener entre 6 y 8 numeros despues del 15
texto = "Mi nombre es María y mi teléfono es 1564232324"
regla_de_busqueda = "15\d{6,8}"
print(re.findall(regla_de_busqueda,texto))

# En realidad los telefonos no arrancan siempre con 15
# capaz empiezan con 11 si son de buenos aires por ejemplo
texto = "Mi nombre es Carlos y mi teléfono es 1164232324"
regla_de_busqueda = "(?:15|11)\d{6,8}"
print(re.findall(regla_de_busqueda,texto))

# En realidad los telefonos pueden tener un guión o espacio a parte de números
texto = "Mi nombre es asfasfeaf33 y mi teléfono es 116423-2324"
regla_de_busqueda = "(?:15|11)[0-9\s-]{6,10}"
print(re.findall(regla_de_busqueda,texto))

# b- Como extraer el mes de un texto
texto = "REPORTE DE PERFOMANCE - MES DE JUNIO"
regla_de_busqueda = "(MES DE (?:JULIO|AGOSTO|JUNIO))"
print(re.findall(regla_de_busqueda,texto))

['1564232324']
['1564232324']
['1164232324']
['116423-2324']
['MES DE JUNIO']


In [ ]:
# ¿Cómo hago que pare de buscar el operador * ?
text = "me llamo pedro. me gusta el rock."
regla_de_busqueda_no_greedy = "(.*?)\."
regla_de_busqueda_greedy = "(.*)\."
print(re.findall(regla_de_busqueda_no_greedy,text))
print(re.findall(regla_de_busqueda_greedy,text))

['me llamo pedro', ' me gusta el rock']
['me llamo pedro. me gusta el rock']


In [ ]:
# python utiliza la libreria llamada re para todo lo relacionado a regular expressions
import re

comentario_de_mercadolibre = 'hola soy aasasda@mariadominguez, me interesa el producto, te dejo mi celu 1565525233, saludos'

def encontrar_telefonos(texto):
    regla_de_busqueda = r'(15[0-9]{8})'
    return re.findall(regla_de_busqueda, texto)

def encontrar_usuarios(texto):
    regla_de_busqueda = r'([a-z0-9]+@[a-zA-Z]+)'
    return re.findall(regla_de_busqueda, texto)

print(encontrar_telefonos(comentario_de_mercadolibre))
print(encontrar_usuarios(comentario_de_mercadolibre))

['1565525233']
['aasasda@mariadominguez']


#### Ejercicio

Usa regex para hacer una función que busque todos los emails en un texto

In [ ]:
def encontrar_emails(texto):
    regla_de_busqueda = r'(?:\d+|\w+)@(?:\d+|\w+).(?:\d+|\w+)'
    return re.findall(regla_de_busqueda,texto)

texto = "Hola te paso mi mail python@hotmail.com, saludos. Si no te funciona mandame a este otro, pedro_2010@yahoo.com"
encontrar_emails(texto)

['python@hotmail.com', 'pedro_2010@yahoo.com']

#### Aplicandolo a la web
##### Ejemplo 1: Usamos regex para extraer los títulos del diario La Prensa.


```html
<h2 class="entry__title"><a href="http://www.laprensa.com.ar/491843-Dilemas-de-la-batalla-cultural-I.note.aspx" target="_self" onclick="javascript:if(typeof(_gaq)!='undefined'){_gaq.push(['_trackEvent', 'Notas', 'Cultura', 'Dilemas de la batalla cultural (I)'])};">Dilemas de la batalla cultural (I)</a></h2>
```


In [ ]:
#Usamos el navegador para identificar la estructura de los datos que queremos extraer y creamos el patrón de búsqueda
regla_de_busqueda = r';">(.+)</a></h2>'

In [ ]:
#Usamos findall para encontrar todas las coincidencias
import re
titles = re.findall(regla_de_busqueda, codigo_html("http://www.laprensa.com.ar/"))

In [ ]:
titles

['Deuda: un respiro en medio de un deterioro sin precedentes',
 'Dilemas de la batalla cultural (II)',
 'Cupos, cupos y más cupos, el sexo del absurdo',
 'El campo digital se expone en San Nicolás',
 'Gracias al Bocha, el campeonato se tiñe de rojo']

#### Ejercicio

#### Modifiquen la regla de búsqueda para que descargue los links a las notas en vez del título

In [ ]:


regla_de_busqueda2 = r'class="entry__title"><a href="(.+?)"'
titulos = re.findall(regla_de_busqueda2, codigo_html("http://www.laprensa.com.ar/"))
titulos

['http://www.laprensa.com.ar/495669-Fernandez-acuso-a-Macri-y-Vidal-de-no-ejecutar-obras-para-prevenir-inundaciones.note.aspx',
 'http://www.laprensa.com.ar/495675-Renuncio-Maria-Eugenia-Bielsa-como-ministra-de-Desarrollo-Territorial-y-Habitat.note.aspx',
 'http://www.laprensa.com.ar/495670-Tenia-miedo-de-el-lo-habia-notado-extrano-dijo-la-bailarina-atacada-en-Belgrano.note.aspx',
 'http://www.laprensa.com.ar/495662-Acusan-a-Macri-y-Aguad-de-encubrir-el-hundimiento-del-submarino-ARA-San-Juan.note.aspx',
 'http://www.laprensa.com.ar/495647-Punto-muerto.note.aspx',
 'http://www.laprensa.com.ar/495660-Murio-el-principe-Khalifa-primer-ministro-de-Bahrein-durante-medio-siglo.note.aspx',
 'http://www.laprensa.com.ar/495664-Cuando-Chicago-mando-a-Boca-al-Matadero.note.aspx',
 'http://www.laprensa.com.ar/495646-El-payaso-de-hoy-se-parece-al-mimo.note.aspx',
 'http://www.laprensa.com.ar/495652-Maxima-cancelo-sus-vacaciones-a-la-Argentina-tras-las-criticas-por-viajar-en-plena-pandemia-a-Grecia.n

### Método 2: BeautifulSoup
Esta librería provee un parser de html, o sea un programa que entiende el código, permitiendonos hacer consultas mas sofisticadas de forma simple, por ejemplo "buscame todos los titulos h2 del sitio".

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(codigo_html("http://www.laprensa.com.ar/"), 'html.parser')
for title in soup.find_all("h2",class_="entry__title"):
    print(title.find("a").text)

Fernández acusó a Macri y Vidal de no ejecutar obras para prevenir inundaciones
Renunció María Eugenia Bielsa como ministra de Desarrollo Territorial y Hábitat
“Tenía miedo de él, lo había notado extraño", dijo la bailarina atacada en Belgrano
Acusan a Macri y Aguad de encubrir el hundimiento del submarino ARA San Juan
Punto muerto­
Murió el príncipe Khalifa, primer ministro de Bahréin durante medio siglo
Cuando Chicago mandó a Boca al Matadero
"El payaso de hoy se parece al mimo''­
Máxima canceló sus vacaciones a la Argentina tras las críticas por viajar en plena pandemia a Grecia


#### Ejemplo 2: Cortazar

In [ ]:
url = 'http://ciudadseva.com/autor/julio-cortazar/cuentos/'
import re
response = requests.get(url)
codigo_html_crudo = response.text

In [ ]:
# Creo carpeta donde voy a guardar los cuentos
!mkdir -p multimedia/cortazar/

for s_url in re.findall(r'(https://ciudadseva.com/texto/.+/)', codigo_html_crudo):
    cuento = requests.get(s_url)
    codigo_html_interpretado = BeautifulSoup(cuento.content, 'html.parser')
    for elem in codigo_html_interpretado.findAll("div", { "class" : "text-justify" })[:2]:
        cuento = elem.text
    
    # Asi podemos guardar los resultados
    nombre_del_archivo = s_url.split('/')[-2]
    with open (f"multimedia/cortazar/{nombre_del_archivo}.txt", 'w') as out:
        out.write(cuento)

## Práctica: Mercadolibre

Descargá y calculá el promedio de los precios que aparecen en la primer página de mercado libre al buscar gibson

In [ ]:
precios

'152.271'

In [1]:
import requests
import re

def precios_gibson():
    url = "https://listado.mercadolibre.com.ar/gibson"
    soup = BeautifulSoup(codigo_html(url), 'html.parser')
    prices = []
    # COMPLETAR
    # SOLUCION:
    for elem in soup.findAll('span', {'class': "price-tag-fraction"}):
        prices.append(elem.text.replace('.', ''))
    
    return list(map(float, prices))


precios2 = precios_gibson()
precios2
#import numpy as np
#print(f"El precio promedio es {int(np.nanmean(precios))}")

NameError: ignored

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import os

In [ ]:
def descargar_letras(artista):
    # COMPLETAR
    url = "https://www.letras.com"
    url_final = url+"/"+artista
    !mkdir -p "multimedia/cancion/" + artista
    soup = BeautifulSoup(codigo_html(url_final), 'html.parser')
    lista_links = []
    for elem in soup.findAll('a', {'class': "song-name"}):
        link_cancion = url+elem['href']
        prueba = BeautifulSoup(codigo_html(link_cancion), 'html.parser')
        for letracancion in prueba.findAll("div", { "class" : "cnt-letra p402_premium" })[:2]:
          letrafinal = letracancion.text
        with open (f"multimedia/cancion/{elem.text}.txt", 'w') as out:
          out.write(letrafinal)
    return





artista = "callejeros"
salida = descargar_letras(artista)


In [ ]:
    url = "https://www.letras.com"
    soup = BeautifulSoup(codigo_html(url), 'html.parser')
    lista_links = []
    for elem in soup.findAll('a', {'class': "song-name"}):
        lista_links.append(elem.text)
    
    lista_links

[]

In [ ]:
url_final = url+"/"+artista
url_final

'https://www.letras.com/spinetta'

In [ ]:
    artista = "callejeros"
    url = "https://www.letras.com"
    url_final = url+"/"+artista
    !mkdir -p "multimedia/cancion/" + artista
    soup = BeautifulSoup(codigo_html(url_final), 'html.parser')
    lista_links = []
    for elem in soup.findAll('a', {'class': "song-name"}):
        link_cancion = lista_links.append(url+elem['href'])
  

In [ ]:
letrafinal

' Siempre tuve un defecto, no se decir que noy pasadas las 4, mi cara era un errorentrancado como un idiota fuicon el moño en la cabezabuscando en esas tetas el calor para poder sobrevivira la agonía de la nochea la desdicha de sentirme un mercenario del alcoholde alejarme entre los gritos y los tragosotra vez solo, un perdedorsalimos de aquella histeria hacia otro lugarleyendo de los colmillos de la soledadregalado, ofreci el sabor de aquellosque en albergue se hacen tibiossi no llegan al orgasmo ganador, que ganadorfue ahi que comprobe que siempre puede haber algo peorfue asi que comprobe que la angustia es prima de la desesperacióny que a veces, tal vez, estar solo es mejory que al cielo no se llega nunca de a dosme quede dormido y con ganas de mearno existe peor remedio que la enfermedadfin de turnolo molesto señor a este animal nocturno la mañana lo encontróal huir, sin higado, sin techo y sin amorfue ahi que comprobe que siempre puede haber algo mejorfue asi que comprobe que la a

## ¿Entonces me puedo descargar todo internet?

En la próximas clases veremos algunas limitaciones de este método y sus alternativas. Mas allá de eso es importante ponerse de vez en cuando en el lugar del sitio del cual estamos descargando datos.


Muchas veces las páginas web obtienen sus ingresos a partir del uso de usuarios tradicionales (humanos) pero no de los scrapers (máquinas). Por lo que estos no generan ganancias al sitio y encima pueden causar congestión en los servidores (Pudiendo causar incluso la rotura del sitio similar a lo que pasa con los [ataques DDOS](https://es.wikipedia.org/wiki/Ataque_de_denegaci%C3%B3n_de_servicio)).

Por esta razón los sitios webs suelen tener una pagina [/robots.txt](https://es.wikipedia.org/wiki/Est%C3%A1ndar_de_exclusi%C3%B3n_de_robots) donde especifican que tipo de scrapeo prefieren evitar para poder mantener su sitio funcionando correctamente sin problemas.

Pueden ver, como ejemplos:

- https://www.google.com/robots.txt
- https://en.wikipedia.org/robots.txt